In [1]:
%matplotlib inline
from genetic_oscillator_benchmarking_futures_four_core import *

/home/twright/Documents/PhD/Notebooks/notebooks/Genetic Oscillator/genetic_oscillator_benchmarking_fixed.py:102: PyGIWarning: Notify was imported without specifying a version first. Use gi.require_version('Notify', '0.7') before import to ensure that the right version gets loaded.
  from gi.repository import Notify


## Properties

In [2]:
P = Atomic(var("x6") > 1)
P

Atomic(x6 > 1)

In [3]:
def prop(k):
    return 125**2*(var("x4") - 0.003)**2 + 3*(var("x6") - 0.5)**2 < k


def prop1(k):
    return 125**2*(var("x4") - 0.007)**2 + 3*(var("x6") - 0.5)**2 < k

In [4]:
Qprop = prop1(0.032)

In [5]:
Q = Atomic(Qprop); Q

Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032)

In [6]:
I = RIF(3, 3.5)

## Launch Runs

In [7]:
num_runs = 20
duration = 1.5

In [8]:
our_property = P | G(RIF(3, 3.5), Q)

In [9]:
our_property_reversed = G(RIF(0,1), G(RIF(3, 3.5), Q) | P)

In [10]:
def run_name(**kwargs):
    res = 'masked' if kwargs.get('use_masks', False) else 'unmasked'
    if not kwargs.get('tentative_unpreconditioning', True):
        res = f'untentative_{res}' 
    if not kwargs.get('selective_unpreconditioning', True):
        res = f'unselective_{res}' 
    if kwargs.get('symbolic_composition', False):
        res = f'symbolic_{res}' 
    if kwargs.get('unprecondition_upfront', False):
        res = f'upfront_{res}'
    if kwargs.get('reverse', False):
        res = f'{res}_reversed'
    return res

In [11]:
option_sets = [
    dict(use_masks=mask, symbolic_composition=symb,
         unprecondition_upfront=upfront, reverse=reverse)
    for reverse in [False] # [True, False]
    for mask in [True, False]
    for symb in [True, False]
    for upfront in [True, False]
]; option_sets

[{'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': False,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': False,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': True,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': False,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': False,
  'unprecondition_upfront': False,
  'reverse': False}]

In [12]:
# option_sets = [
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'tentative_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': False,
#   'unprecondition_upfront': True,
#   'selective_unpreconditioning': False,
#   'reverse': False},
#  {'use_masks': False,
#   'symbolic_composition': False,
#   'unprecondition_upfront': False,
#   'reverse': False},
#  {'use_masks': False,
#   'symbolic_composition': True,
#   'unprecondition_upfront': False,
#   'reverse': False},
#  {'use_masks': True,
#   'symbolic_composition': True,
#   'unprecondition_upfront': True,
#   'reverse': False},
# ]

In [13]:
our_property_reversed

G([0 .. 1], Or([G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032)), Atomic(x6 > 1)]))

In [14]:
option_sets

[{'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': False,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': True,
  'symbolic_composition': False,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': True,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': True,
  'unprecondition_upfront': False,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': False,
  'unprecondition_upfront': True,
  'reverse': False},
 {'use_masks': False,
  'symbolic_composition': False,
  'unprecondition_upfront': False,
  'reverse': False}]

In [15]:
results = {
    (name := run_name(**option_set)): gen_results(
        num_runs,
        our_property_reversed
            if option_set.get('reverse', False)
            else our_property,
        duration,
        task_description=name,
        **option_set,
    )
    for option_set in option_sets
}

/usr/lib64/python3.8/multiprocessing/queues.py:116: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  return _ForkingPickler.loads(res)
/usr/lib64/python3.8/multiprocessing/queues.py:116: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  return _ForkingPickler.loads(res)
/usr/lib64/python3.8/multiprocessing/queues.py:116: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  return _ForkingPickler.loads(res)


phi variables sfs = {x4, x6}
phi variables sfs = {x4, x6}
phi variables sfs = {x4, x6}
system sfs = System(R, (x1, x2, x3, x4, x5, x6, x7, x8, x9), [[0.97999999999999998 .. 1.0200000000000001], [1.2799999999999998 .. 1.3200000000000001], [0.079999999999999987 .. 0.12000000000000001], [0.079999999999999987 .. 0.12000000000000001], [0.079999999999999987 .. 0.12000000000000001], [1.2799999999999998 .. 1.3200000000000001], [2.4799999999999999 .. 2.5200000000000001], [0.57999999999999996 .. 0.62000000000000011], [1.2799999999999998 .. 1.3200000000000001]], (-0.10000000000000000?*x1*x6 + 50*x3, -x2*x6 + 100*x4, 0.10000000000000000?*x1*x6 - 50*x3, x2*x6 - 100*x4, 0.50000000000000000?*x1 + 5*x3 - 10*x5, -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6, 0.01000000000000000?*x2 + 50*x4 - 0.50000000000000000?*x7, -2*x6*x8 + 0.50000000000000000?*x7 - 0.2000000000000000?*x8 + x9, 2*x6*x8 - x9), varmap=bidict({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6, 'x7

top-level root detection: 1 items in 31.90 seconds
roots = []
domain = [0.0000000000000000 .. 5.0020000000000007]
roots = []
seg_domain = [3.9083802438732977 .. 5.0019999999999998]
roots = []
signal given bool roots: 1 items in 0.12 seconds
Run done in 206.26786192096188 sec
options: {'instrumentor': <flowstar.instrumentation.AggregateMetric object at 0x7fece017d280>, 'print_timings': False, 'use_masks': True, 'symbolic_composition': True, 'unprecondition_upfront': True, 'reverse': False}
ran "Precomposing Contexts" 1 times in 0.0004448890686035156
ran "restoring globals" 16 times in 0.0011069774627685547
ran "Running Flow*" 1 times in 88.19359087944031
ran "prepare" 1 times in 0.09885954856872559
ran "composing flowpipe" 1668 times in 82.32795786857605
ran "unpreconditioning flowpipes" 1 times in 82.37906312942505
ran "capturing globals [on exit]" 16 times in 0.0005624294281005859
ran "toHornerForm" 2 times in 3.5762786865234375e-05
ran "mask intersect check" 3336 times in 0.009911537

Computed 1668 flowpipes in 162.91386074200273 sec
PolyObserver(x6 - 1, <flowstar.reachability.Reach object at 0x7fea5354bca0>, -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6, symbolic_composition=True, symbolic_composition_order=4, tentative_unpreconditioning=True, mask=Mask([0.0000000000000000 .. 5.0020000000000007], [[0.0000000000000000 .. 1.5019999999999998]]))
fprime = -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6
roots(space_domain=None, epsilon=1e-05, verbosity=0)
top-level root detection: 1 items in 1.63 seconds
roots = [[0.90838024387329774 .. 1.2778497471154513]]
domain = [0.0000000000000000 .. 5.0020000000000007]
roots = [[0.90838024387329774 .. 1.2778497471154513]]
seg_domain = [0.0000000000000000 .. 1.5019999999999998]
roots = [[0.90838024387329774 .. 1.2778497471154513]]
signal given bool roots: 1 items in 0.01 seconds
In G.signal
PolyObserver(-15625*x4^2 - 3*x6^2 + 218.7500000000001?*x4 + 3*x6 - 1.483625000000000?

top-level root detection: 1 items in 38.77 seconds
roots = []
domain = [0.0000000000000000 .. 5.0020000000000007]
roots = []
seg_domain = [3.9083802438732977 .. 5.0019999999999998]
roots = []
signal given bool roots: 1 items in 0.10 seconds
Run done in 218.72111497999867 sec
options: {'instrumentor': <flowstar.instrumentation.AggregateMetric object at 0x7fece016f900>, 'print_timings': False, 'use_masks': True, 'symbolic_composition': True, 'unprecondition_upfront': True, 'reverse': False}
ran "Precomposing Contexts" 1 times in 4.482269287109375e-05
ran "restoring globals" 16 times in 0.001317739486694336
ran "Running Flow*" 1 times in 81.19079375267029
ran "prepare" 1 times in 0.10367035865783691
ran "composing flowpipe" 1668 times in 96.69538760185242
ran "unpreconditioning flowpipes" 1 times in 96.75834465026855
ran "capturing globals [on exit]" 16 times in 0.0006825923919677734
ran "toHornerForm" 2 times in 3.218650817871094e-05
ran "mask intersect check" 3336 times in 0.01113629341

...

ran "restoring globals" 16 times in 0.0013871192932128906
ran "Running Flow*" 1 times in 83.04368615150452
ran "capturing globals [on exit]" 16 times in 0.011242151260375977
ran "prepare" 1 times in 0.11242532730102539
ran "toHornerForm" 2 times in 2.86102294921875e-05
ran "mask intersect check" 3336 times in 0.006938934326171875
ran "tentative eval" 3191 times in 3.702676773071289
ran "composing flowpipe" 1375 times in 99.55303907394409
ran "pre retrieve" 1520 times in 0.011704206466674805
ran "whole domain eval" 1520 times in 3.260033369064331
ran "compose_interval_fn for atomic" 1324 times in 0.018883466720581055
ran "post retrieve" 1323 times in 0.03731870651245117
ran "root detection" 1323 times in 7.357861518859863
ran "root amalgamation" 1323 times in 0.005234241485595703
ran "prepare in check" 5 times in 4.3392181396484375e-05
ran "check boolean [unguarded]" 5 times in 0.010250568389892578
ran "check_single_point" 5 times in 0.011021614074707031
ran "observer.check" 5 times in 

fprime = -31250*x2*x4*x6 + 0.600000000000000?*x1*x6^2 + 6*x2*x6^2 + 12*x6^2*x8 + 3125000*x4^2 - 0.3000000000000000?*x1*x6 + 215.75000000000000?*x2*x6 - 300*x3*x6 - 600*x4*x6 - 300*x5*x6 + 6*x6^2 - 6*x6*x8 + 150*x3 - 21575*x4 + 150*x5 - 3*x6
roots(space_domain=None, epsilon=1e-05, verbosity=0)
Computed 1668 flowpipes in 85.6629899990221 sec
PolyObserver(x6 - 1, <flowstar.reachability.Reach object at 0x7fea5354bca0>, -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6, symbolic_composition=False, symbolic_composition_order=4, tentative_unpreconditioning=True, mask=None)
fprime = -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6
roots(space_domain=None, epsilon=1e-05, verbosity=0)
top-level root detection: 1 items in 86.75 seconds
roots = [[1.4639099741417731 .. 1.4640000000000184], [1.4655013774685898 .. 5.0040000000001533]]
domain = [0.0000000000000000 .. 5.0020000000000007]
roots = [[1.4639099741417731 .. 1.4640000000000184], 
 [1.46550

Computed 1668 flowpipes in 87.67309364504763 sec
PolyObserver(x6 - 1, <flowstar.reachability.Reach object at 0x7fea5354bca0>, -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6, symbolic_composition=False, symbolic_composition_order=4, tentative_unpreconditioning=True, mask=None)
fprime = -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6
roots(space_domain=None, epsilon=1e-05, verbosity=0)
top-level root detection: 1 items in 17.50 seconds
roots = [[0.88667270934066744 .. 1.3094835775584244]]
domain = [0.0000000000000000 .. 5.0020000000000007]
roots = [[0.88667270934066744 .. 1.3094835775584244]]
seg_domain = [0.0000000000000000 .. 5.0020000000000007]
roots = [[0.88667270934066744 .. 1.3094835775584244]]
signal given bool roots: 1 items in 0.01 seconds
In G.signal
PolyObserver(-15625*x4^2 - 3*x6^2 + 218.7500000000001?*x4 + 3*x6 - 1.483625000000000?, <flowstar.reachability.Reach object at 0x7fea5354bca0>, -31250*x2*x4*x6 + 0.60000000000

ran "whole domain eval" 1520 times in 3.2618064880371094
ran "compose_interval_fn for atomic" 1324 times in 0.017949342727661133
ran "post retrieve" 1323 times in 0.036368608474731445
ran "root detection" 1323 times in 7.72497820854187
ran "root amalgamation" 1323 times in 0.005337953567504883
ran "prepare in check" 5 times in 4.267692565917969e-05
ran "check boolean [unguarded]" 5 times in 0.011868715286254883
ran "check_single_point" 5 times in 0.012735128402709961
ran "observer.check" 5 times in 0.012989997863769531
ran "Monitoring atomic x6 > 1" 1 times in 17.519054651260376
ran "returning cached composed flowpipe" 147 times in 0.00033473968505859375
ran "Monitoring atomic 15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032" 1 times in 96.02823257446289
ran "Monitoring Signal for x6 > 1 | G([3.0000000000000000 .. 3.5000000000000000], 15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032)" 1 times in 113.64031076431274
phi variables sfs = {x4, x6}
system sfs = System(R, (x1, x2, x3, x4, x5, x6, x

ran "returning cached composed flowpipe" 147 times in 0.0002639293670654297
ran "Monitoring atomic 15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032" 1 times in 78.30844640731812
ran "Monitoring Signal for x6 > 1 | G([3.0000000000000000 .. 3.5000000000000000], 15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032)" 1 times in 92.21322798728943
Computed 1668 flowpipes in 87.8149634420115 sec
PolyObserver(x6 - 1, <flowstar.reachability.Reach object at 0x7fea5354bca0>, -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6, symbolic_composition=False, symbolic_composition_order=4, tentative_unpreconditioning=True, mask=None)
fprime = -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6
roots(space_domain=None, epsilon=1e-05, verbosity=0)
Computed 1668 flowpipes in 82.1319859760406 sec
PolyObserver(x6 - 1, <flowstar.reachability.Reach object at 0x7fea5354bca0>, -0.10000000000000000?*x1*x6 - x2*x6 - 2*x6*x8 + 50*x3 + 100*x4 + 50*x5 - x6, symbolic_composition=

In [22]:
results['upfront_symbolic_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 1.5000000000000000], True)], mask=Mask([0.0000000000000000 .. 1.5000000000000000], [[0.0000000000000000 .. 1.5000000000000000]]))

In [27]:
results['upfront_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 0.88667270934066745], True)], mask=Mask([0.0000000000000000 .. 1.5000000000000000], [[0.0000000000000000 .. 1.5000000000000000]]))

In [23]:
results['symbolic_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 1.5000000000000000], True)], mask=Mask([0.0000000000000000 .. 1.5000000000000000], [[0.0000000000000000 .. 1.5000000000000000]]))

In [24]:
results

{'upfront_symbolic_masked': [{'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': True,
    'reverse': False},
   'future': <Future at 0x7f532107a3a0 state=finished returned tuple>}],
 'symbolic_masked': [{'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': False,
    'reverse': False},
   'future': <Future at 0x7f532107ae80 state=finished returned tuple>}],
 'upfront_masked': [{'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': False,
    

In [32]:
results_finished = {
    k: v
    for k, v in results.items()
    if all(r['future'].done()
           for r in v)
}; results_finished

{'upfront_symbolic_masked': [{'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': True,
    'reverse': False},
   'future': <Future at 0x7fea534b4a60 state=finished returned tuple>},
  {'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': True,
    'reverse': False},
   'future': <Future at 0x7fece0183ac0 state=finished returned tuple>},
  {'prop': Or([Atomic(x6 > 1), G([3.0000000000000000 .. 3.5000000000000000], Atomic(15625*(x4 - 0.007)^2 + 3*(x6 - 0.5)^2 < 0.032))]),
   'duration': 1.5,
   'kwargs': {'use_masks': True,
    'symbolic_composition': True,
    'unprecondition_upfront': True,
    'reve

# Analysis

In [16]:
import pandas as pd

In [22]:
results['masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 0.88667270934066745], True)], mask=Mask([0.0000000000000000 .. 1.5000000000000000], [[0.0000000000000000 .. 1.5000000000000000]]))

In [17]:
results['symbolic_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 1.5000000000000000], True)], mask=Mask([0.0000000000000000 .. 1.5000000000000000], [[0.0000000000000000 .. 1.5000000000000000]]))

In [18]:
results['symbolic_unmasked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 1.5000000000000000], True)], mask=None)

In [19]:
results['upfront_symbolic_masked'][0]['future'].result()[0]

Signal([0.0000000000000000 .. 1.5000000000000000], [([0.0000000000000000 .. 1.5000000000000000], True)], mask=Mask([0.0000000000000000 .. 1.5000000000000000], [[0.0000000000000000 .. 1.5000000000000000]]))

In [20]:
results['symbolic_masked'][0]['future'].result()[2].print()

ran "Precomposing Contexts" 1 times in 5.9604644775390625e-05
ran "restoring globals" 16 times in 0.0011548995971679688
ran "Running Flow*" 1 times in 79.37443852424622
ran "capturing globals [on exit]" 16 times in 0.010451316833496094
ran "prepare" 1 times in 0.10670137405395508
ran "toHornerForm" 2 times in 3.600120544433594e-05
ran "mask intersect check" 3336 times in 0.01171565055847168
ran "tentative eval" 867 times in 0.892289400100708
ran "composing flowpipe" 511 times in 29.783427476882935
ran "pre retrieve" 511 times in 0.0035889148712158203
ran "whole domain eval" 511 times in 0.8653879165649414
ran "observable_hf" 509 times in 32.31979537010193
ran "post retrieve" 508 times in 32.20018029212952
ran "root detection" 508 times in 0.014609336853027344
ran "root amalgamation" 508 times in 0.0011169910430908203
ran "prepare in check" 3 times in 4.100799560546875e-05
ran "check boolean [unguarded]" 3 times in 0.13669610023498535
ran "check_single_point" 3 times in 0.14573550224304

In [21]:
results['upfront_symbolic_masked'][0]['future'].result()[2].print()

ran "Precomposing Contexts" 1 times in 0.0003192424774169922
ran "restoring globals" 16 times in 0.0015614032745361328
ran "Running Flow*" 1 times in 93.03791236877441
ran "prepare" 1 times in 0.0982513427734375
ran "composing flowpipe" 1668 times in 99.92983603477478
ran "unpreconditioning flowpipes" 1 times in 99.99059748649597
ran "capturing globals [on exit]" 16 times in 0.0008099079132080078
ran "toHornerForm" 2 times in 3.266334533691406e-05
ran "mask intersect check" 3336 times in 0.010847806930541992
ran "returning cached composed flowpipe" 868 times in 0.0017786026000976562
ran "pre retrieve" 867 times in 0.004515171051025391
ran "whole domain eval" 867 times in 1.530977725982666
ran "observable_hf" 509 times in 34.30483531951904
ran "post retrieve" 508 times in 34.207629680633545
ran "root detection" 508 times in 0.006228446960449219
ran "root amalgamation" 508 times in 0.00110626220703125
ran "prepare in check" 3 times in 3.24249267578125e-05
ran "check boolean [unguarded]" 

In [23]:
cols = [
    ('Flow*' , 'Running Flow*'),
    ('unpreconditioning', 'unpreconditioning flowpipes'),
    ('P', f'Monitoring atomic {P}'),
    ('Q',  f'Monitoring atomic {Q}'),
    ('G(P | G(Q))', [
        f'Monitoring Signal for {our_property}',
        f'Monitoring Signal for {our_property_reversed}',
    ]),
    ('tentative eval', 'tentative eval'),
    ('eval', 'whole domain eval'),
    ('root detection', 'root detection'),
    ('composing flowpipe', 'composing flowpipe'),
    ('composing atomic', 'post retrieve'),
]
cols_short, cols_full = zip(*cols)

## Raw results

In [24]:
total_times(
    results['masked'],
    cols_full,
    cols_short,
)

,Flow*,unpreconditioning,P,Q,G(P | G(Q)),tentative eval,eval,root detection,composing flowpipe,composing atomic,Total
0,81.577519,0.0,14.941538,22.739830,37.767062,0.803705,0.972221,1.888693,33.879407,0.013078,119.473181
1,81.550241,0.0,12.646003,23.144158,35.876606,0.808836,0.914713,1.817426,32.111093,0.013435,117.565685
2,80.287268,0.0,12.410721,22.679352,35.194086,0.795899,0.921575,1.812706,31.424872,0.012904,115.612884
3,81.127772,0.0,13.805287,22.741383,36.639727,0.834439,0.935657,1.851138,32.789686,0.012759,117.899158
4,81.629049,0.0,13.106065,22.793790,35.999285,0.822480,0.929474,1.839120,32.174984,0.012654,117.759616
5,81.131922,0.0,15.553214,22.887431,38.538366,0.803864,0.970848,1.875196,34.643990,0.013813,119.799750
6,81.619814,0.0,12.418635,22.833143,35.335904,0.822291,0.918149,1.816924,31.556714,0.013132,117.081026
7,80.919712,0.0,12.125612,23.076970,35.300754,0.817282,0.920074,1.831319,31.499404,0.013385,116.347507
8,81.498490,0.0,12.582130,23.010745,35.681556,0.808687,0.930457,1.832580,31.872512,0.013372,117.329733
9,81.088065,0.0,21.717935,22.875728,44.694759,0.803568,1.081786,2.020348,40.542701,0.014614,125.925223


In [25]:
total_times(
    results['upfront_masked'],
    cols_full,
    cols_short,
)

,Flow*,unpreconditioning,P,Q,G(P | G(Q)),tentative eval,eval,root detection,composing flowpipe,composing atomic,Total
0,80.635735,84.364605,1.348816,1.733531,3.180110,0.0,1.382014,1.594999,84.300666,0.008737,168.313319
1,80.746475,84.530584,1.376862,1.815471,3.292842,0.0,1.424166,1.653790,84.465912,0.009866,168.694748
2,80.366453,84.353699,1.365047,1.725665,3.174395,0.0,1.403373,1.590424,84.287584,0.007816,168.016961
3,80.139054,84.544859,1.351013,1.745464,3.184322,0.0,1.389277,1.604048,84.481232,0.008980,167.993093
4,79.994590,84.623093,1.370438,1.747370,3.201414,0.0,1.389056,1.606751,84.560092,0.009376,167.942888
5,80.539772,84.264632,1.352419,1.791339,3.246136,0.0,1.418306,1.630057,84.198357,0.008315,168.181063
6,81.089103,86.415225,1.420552,1.738593,3.253595,0.0,1.426603,1.625738,86.348413,0.008900,170.890522
7,79.822531,85.073135,1.365084,1.775903,3.257139,0.0,1.399069,1.625584,85.010058,0.009770,168.279818
8,80.891425,84.534699,1.342982,1.780548,3.214932,0.0,1.401330,1.603430,84.468933,0.008222,168.783576
9,80.504325,84.490010,1.352617,1.752857,3.195294,0.0,1.391312,1.603244,84.425216,0.008927,168.316097


In [26]:
_['Flow*'].std()

0.4829981669057681

## Export Data

In [27]:
import pickle

In [28]:
total_timess = {
    s: total_times(res, cols_full, cols_short)
    for s, res in results.items()
}

In [29]:
total_timess

{'upfront_symbolic_masked':         Flow*  unpreconditioning         P          Q  G(P | G(Q))  \
 0   93.037912          99.990597  1.414934  34.554525    37.692570   
 1   88.193591          82.379063  1.559907  32.023099    35.572541   
 2   88.107918          82.477291  1.689444  32.506632    36.335852   
 3   79.014492          83.778572  1.651174  32.871214    34.623089   
 4   81.190794          96.758345  1.654472  38.882716    40.651627   
 5   78.907573          84.221171  1.422059  32.164594    33.699974   
 6   78.797375          83.557400  1.292628  30.061932    31.437836   
 7   79.493376          91.330907  1.426431  31.124020    32.651320   
 8   78.945960          88.648969  1.345500  31.837876    33.283945   
 9   83.100497          84.396170  1.403908  31.902435    33.403642   
 10  79.347101          84.458356  1.476000  31.345170    32.915882   
 11  78.930231          83.791966  1.371456  35.482217    36.946004   
 12  78.831713          83.363511  1.287713  29.93

In [30]:
results_finished

NameError: name 'results_finished' is not defined

In [33]:
for s, res in results_finished.items():
    with open(f'data/results_fixed_{s}1.pkl', 'wb') as file:
        pickle.dump([f.result() for f in futures(res)],
                    file)

In [34]:
for s, d in total_timess.items():
    d.to_csv(f'data/benchmark_fixed_{s}1.csv', index_label=False)

In [35]:
with open('data/benchmark_fixed_metadata1.pkl', 'wb') as file:
    pickle.dump((cols_full, cols_short, list(results_finished.keys())), file)